In [4]:
import nengo
import nengo_dl
import tensorflow as tf
from models import *
from datasets import *

inp = tf.keras.Input(shape=(16,), name='glucose')
lam = tf.keras.layers.Lambda(inputMask)(inp)
enc = tf.keras.layers.Dense(3, activation='sigmoid', name='encoder')(lam)
dec = tf.keras.layers.Dense(16, activation='sigmoid', name='decoder')(enc)
autoencoder = tf.keras.Model(inputs=inp, outputs=dec)

converter = nengo_dl.Converter(autoencoder)

with nengo.Network() as net:
	nengo_dl.configure_settings(stateful=False)

	a = nengo.Node(output=[1])
	b = nengo.Ensemble(n_neurons=100, dimensions=1)
	nengo.Connection(a, b)
	p = nengo.Probe(b)

X = load_glucose_data()
print("Loaded", len(X), "samples")

with nengo_dl.Simulator(converter.net, minibatch_size=5, device='/gpu:0') as sim:
	sim.compile(
		optimizer=tf.optimizers.Adam(0.001),
		loss=tf.losses.MeanAbsoluteError,
		metrics=[tf.metrics.RootMeanSquaredError]
	)
	sim.fit(
		{converter.inputs[inp]: X},
		{converter.outputs[dec]: X},
		# validation_data=(
		#     {converter.inputs[inp]: test_images},
		#     {converter.outputs[dense]: test_labels},
		# ),
		epochs=2, batch_size=32
	)

	sim.save_params("./keras_to_snn_params")



Loaded 48430 samples
|                     Building network (0%)                    | ETA:  --:--:--
Build finished in 0:00:00
|#                         Optimizing graph                           | 0:00:00
|#             Optimizing graph: operator simplificaton               | 0:00:00
Optimizing graph: operator simplificaton finished in 0:00:00
|#                Optimizing graph: merging operators                 | 0:00:00
Optimizing graph: merging operators finished in 0:00:00
|#                Optimizing graph: ordering signals                  | 0:00:00
Optimizing graph: ordering signals finished in 0:00:00
|#                Optimizing graph: creating signals                  | 0:00:00
Optimizing graph: creating signals finished in 0:00:00
Optimization finished in 0:00:00
|#                        Constructing graph                          | 0:00:00
|           Constructing graph: pre-build stage (0%)           | ETA:  --:--:--
Constructing graph: pre-build stage finished in 0:00:

TypeError: 'property' object is not iterable